# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Ana Sofía Avila Gálvez  |
| **Fecha**      | 11/09/25  |
| **Expediente** | 745247  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente.

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("Advertising.csv")
data

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [4]:
#Sin penalización
import statsmodels.api as sm
unos=np.ones([200,1])
x = data[["TV", "radio", "newspaper"]]
y = data[["sales"]]

X = sm.add_constant(x)
ols = sm.OLS(y, X)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        19:26:45   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
radio          0.1885      0.009     21.893      0.000       0.172       0.206
newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
r2=results.rsquared
r2

np.float64(0.8972106381789522)

In [5]:
#Con penalización L2
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
X = data[["TV", "radio", "newspaper"]]
y = data["sales"]

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_scaled, y)

Ridge()

In [8]:
ridge.intercept_
ridge.coef_

array([ 3.90021344,  2.77691168, -0.0160149 ])

In [9]:
r2= ridge.score(X_scaled, y)
r2

0.8971891437493419

In [10]:
#Con penalización L1
from sklearn.linear_model import Lasso
X = data[["TV", "radio", "newspaper"]]
y = data["sales"]

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [12]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_scaled, y)

Lasso(alpha=0.1)

In [13]:
lasso.intercept_
lasso.coef_

array([3.82360787, 2.68932395, 0.        ])

In [14]:
r2=lasso.score(X_scaled, y)
r2

0.896494232900825

P-Values

In [15]:
#Sin penalización
pvalues=results.pvalues
pvalues

,0
const,1.267295e-17
TV,1.509960e-81
radio,1.505339e-54
newspaper,8.599151e-01


In [16]:
# Bootstrapping para reentrenar el modelo y estimar la distribución de cada coeficiente
n_bootstraps = 1000
coefs = []

for _ in range(n_bootstraps):
    idx = np.random.choice(len(y), len(y), replace=True)
    X_resample = X_scaled[idx]
    y_resample = y[idx]
    ridge.fit(X_resample, y_resample)
    coefs.append(ridge.coef_)

In [17]:
coefs = np.array(coefs)
coefs

array([[ 3.96180489,  2.72321339,  0.13547784],
       [ 3.5835629 ,  2.91822619,  0.05054388],
       [ 3.77442285,  2.85566965, -0.04848969],
       ...,
       [ 3.93520091,  2.86241331, -0.07985738],
       [ 4.03427232,  2.85876777, -0.20866832],
       [ 4.07088992,  2.68198304,  0.03706813]])

In [18]:
coef_means = coefs.mean(axis=0)
coef_stds = coefs.std(axis=0)
(coef_means, coef_stds)

(array([ 3.88227291,  2.78598691, -0.01415721]),
 array([0.17153342, 0.15983372, 0.13589883]))

In [19]:
t_stats = coef_means / coef_stds
t_stats

array([22.63274957, 17.4305326 , -0.1041746 ])

Comparación

In [23]:
#OLS
ols_params = ols.params.values
ols_pvalues = ols.pvalues.values
ols_r2 = ols.rsquared

In [24]:
#RIDGE
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

ridge = Ridge(alpha=1.0)
ridge.fit(X_scaled, y)

ridge_params = np.insert(ridge.coef_, 0, ridge.intercept_)  # intercepto + coef
ridge_r2 = ridge.score(X_scaled, y)

In [25]:
#LASSO
lasso = Lasso(alpha=0.1)
lasso.fit(X_scaled, y)

lasso_params = np.insert(lasso.coef_, 0, lasso.intercept_)
lasso_r2 = lasso.score(X_scaled, y)

In [26]:
#Tabla comparación
results_df = pd.DataFrame({
    "OLS_params": ols_params,
    "OLS_pvalues": ols_pvalues,
    "Ridge_params": ridge_params,
    "Lasso_params": lasso_params
}, index=["Intercepto", "TV", "Radio", "Newspaper"])

results_df

,OLS_params,OLS_pvalues,Ridge_params,Lasso_params
Intercepto,2.938889,1.267295e-17,14.022500,14.022500
TV,0.045765,1.509960e-81,3.900213,3.823608
Radio,0.188530,1.505339e-54,2.776912,2.689324
Newspaper,-0.001037,8.599151e-01,-0.016015,0.000000


In [28]:
#R2
ols_r2, ridge_r2, lasso_r2

(np.float64(0.8972106381789522), 0.8971891437493419, 0.896494232900825)